In [17]:
import cv2
import numpy as np
from mtcnn import MTCNN

In [18]:
# Load MTCNN face detector
detector = MTCNN()

In [19]:
# Load pre-trained models for age and gender detection
age_weights = "age_net.caffemodel"
age_config = "age_deploy.prototxt"
age_net = cv2.dnn.readNet(age_weights, age_config)

gender_weights = "gender_net.caffemodel"
gender_config = "gender_deploy.prototxt"
gender_net = cv2.dnn.readNet(gender_weights, gender_config)

# Model parameters
age_list = ['(0-5)', '(6-10)', '(11-15)', '(16-20)', '(21-30)', '(31-45)', '(46-60)', '(61-80)', '(81-100)']
gender_list = ['Male', 'Female']
model_mean = (104, 117, 123)

# Load and process the image
image_path = r"E:\OPEN CV PROJECT\Age and Gender Project-Open CV\images\images (4).jpeg"
img = cv2.imread(image_path)

if img is None:
    print("Error: Image not found or unable to read the image.")
    exit()

In [20]:
# Convert to RGB for MTCNN
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect faces using MTCNN
faces = detector.detect_faces(img_rgb)

if len(faces) == 0:
    print("No faces detected.")
    exit()

# Process each detected face
for face in faces:
    x, y, w, h = face['box']  # Extract bounding box
    x, y = max(0, x), max(0, y)  # Ensure coordinates are within bounds
    x1, y1 = x + w, y + h
    x1, y1 = min(img.shape[1], x1), min(img.shape[0], y1)

    # Expand the bounding box to include hair and a bit of surrounding area
    margin = int(0.2 * h)  # 20% of the height as margin
    x = max(0, x - margin)
    y = max(0, y - margin)
    x1 = min(img.shape[1], x1 + margin)
    y1 = min(img.shape[0], y1 + margin)

    # Crop the face
    face_img = img[y:y1, x:x1]

    # Check if the cropped face is valid
    if face_img.size == 0:
        print("Error: Cropped face is invalid.")
        continue

    # Resize the face to the input size required by the model
    try:
        face_resized = cv2.resize(face_img, (227, 227))
    except Exception as e:
        print(f"Error resizing face: {e}")
        continue

    # Normalize pixel values
    blob = cv2.dnn.blobFromImage(face_resized, 1.0, (227, 227), model_mean, swapRB=False, crop=False)
    
    # Predict gender
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_list[np.argmax(gender_preds[0])]

    # Predict age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = age_list[np.argmax(age_preds[0])]

    # Annotate the image
    label = f"{gender}, {age}"
    text_scale = 0.5  # Reduced text scale
    text_thickness = 1  # Reduced thickness
    text_color = (0, 255, 0)

    # Adjust the position to ensure visibility
    y_text = y - 10 if y - 10 > 10 else y + 10
    cv2.rectangle(img, (x, y), (x1, y1), (255, 0, 0), 2)
    cv2.putText(img, label, (x, y_text), cv2.FONT_HERSHEY_SIMPLEX, text_scale, text_color, text_thickness)

In [21]:
# Display the annotated image only
cv2.imshow("Age and Gender Detection", img)

# Wait for 3 seconds and then close the window automatically
cv2.waitKey(3000)
cv2.destroyAllWindows() 

# Ensure the window closes properly
if cv2.getWindowProperty("Age and Gender Detection", cv2.WND_PROP_VISIBLE) < 1:
    cv2.destroyAllWindows()
else:
    cv2.destroyAllWindows()